In [3]:
Year = 202408
File = "Disbursement & Repayment August 2024"
Location = r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\\"+str(Year)+"\\Finance"

#control + F + dropdown new to search bar to find and replace 
import pandas as pd
import numpy as np
import xlsxwriter

#warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan

Dis_isl = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Disbursement Islamic Aug24', header=8) # skiprows = 2, nrows=18,  usecols= 'A:D'
Rep_Isl = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Repayment Islamic Aug24', header=8) # skiprows = 2, nrows=18,  usecols= 'A:D'
Dis_Conv = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Disbursement Conv Aug24', header=8) # skiprows = 2, nrows=18,  usecols= 'A:D'
Rep_Conv = pd.read_excel(str(Location)+"\\"+str(File)+".xlsx", sheet_name='Repayment Conv Aug24', header=8) # skiprows = 2, nrows=18,  usecols= 'A:D'


#repayment is Cost Payment/Principal Repayment (Facility Currency)
#+ve kn amount

#disbursement is Disbursement/Drawdown (Facility Currency)

LDB_prev = pd.read_excel(r"C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\\"+str(Year-1)+"\\Loan Database as Jul 2024 (python) - Final.xlsx", sheet_name='Export Worksheet', header=1) # skiprows = 2, nrows=18,  usecols= 'A:D'


In [4]:
Dis_isl.columns = Dis_isl.columns.str.replace("\n", "_")
Dis_isl.columns = Dis_isl.columns.str.replace(" ", "")

Rep_Isl.columns = Rep_Isl.columns.str.replace("\n", "_")
Rep_Isl.columns = Rep_Isl.columns.str.replace(" ", "")

Dis_Conv.columns = Dis_Conv.columns.str.replace("\n", "_")
Dis_Conv.columns = Dis_Conv.columns.str.replace(" ", "")

Rep_Conv.columns = Rep_Conv.columns.str.replace("\n", "_")
Rep_Conv.columns = Rep_Conv.columns.str.replace(" ", "")


In [5]:
Dis_isl_1 = Dis_isl.iloc[np.where((Dis_isl['Unnamed:1']=="**")&(~Dis_isl.Account.isin(['Account']))&~(Dis_isl.Account.isna()))].fillna(0).groupby(['Account','Curr.'])[['Amtinloc.cur.','AmountinDC']].sum().reset_index()
Rep_Isl_1 = Rep_Isl.iloc[np.where((Rep_Isl['Unnamed:1']=="**")&(~Rep_Isl.Account.isin(['Account']))&~(Rep_Isl.Account.isna()))].fillna(0).groupby(['Account','Curr.'])[['Amtinloc.cur.','AmountinDC']].sum().reset_index()
Dis_Conv_1 = Dis_Conv.iloc[np.where((Dis_Conv['Unnamed:1']=="**")&(~Dis_Conv.Account.isin(['Account']))&~(Dis_Conv.Account.isna()))].fillna(0).groupby(['Account','Curr.'])[['Amtinloc.cur.','AmountinDC']].sum().reset_index()
Rep_Conv_1 = Rep_Conv.iloc[np.where((Rep_Conv['Unnamed:1']=="**")&(~Rep_Conv.Account.isin(['Account']))&~(Rep_Conv.Account.isna()))].fillna(0).groupby(['Account','Curr.'])[['Amtinloc.cur.','AmountinDC']].sum().reset_index()

Dis_isl_1['Type_of_Financing'] = 'Islamic'
Rep_Isl_1['Type_of_Financing'] = 'Islamic'
Dis_Conv_1['Type_of_Financing'] = 'Conventional'
Rep_Conv_1['Type_of_Financing'] = 'Conventional'

Disbursement = pd.concat([Dis_isl_1,Dis_Conv_1])
Repayment = pd.concat([Rep_Isl_1,Rep_Conv_1])

Disbursement.rename(columns={'AmountinDC': 'Disbursement_Drawdown_Facility_Currency',
                             'Amtinloc.cur.':'Disbursement_Drawdown_MYR'},inplace=True)

Repayment.rename(columns={'AmountinDC': 'Cost_Payment_Principal_Repayment_Facility_Currency',
                          'Amtinloc.cur.':'Cost_Payment_Principal_Repayment_MYR'},inplace=True)

Repayment['Cost_Payment_Principal_Repayment_Facility_Currency'] = -1*Repayment['Cost_Payment_Principal_Repayment_Facility_Currency']
Repayment['Cost_Payment_Principal_Repayment_MYR'] = -1*Repayment['Cost_Payment_Principal_Repayment_MYR']

merge = Disbursement.fillna(0).merge(Repayment.fillna(0),on=['Account','Curr.','Type_of_Financing'],how='outer')
merge.fillna(0, inplace=True)

C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_6924\634631472.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Dis_isl_1 = Dis_isl.iloc[np.where((Dis_isl['Unnamed:1']=="**")&(~Dis_isl.Account.isin(['Account']))&~(Dis_isl.Account.isna()))].fillna(0).groupby(['Account','Curr.'])[['Amtinloc.cur.','AmountinDC']].sum().reset_index()
C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_6924\634631472.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Rep_Isl_1 = Rep_Isl.iloc[np.where((Rep_Isl['Unnamed:1']=="**")&(~Rep_Isl.Account.isin(['Account']))&~(R

In [7]:
sum(Dis_isl_1['Amtinloc.cur.'])

127525201.88

In [5]:
#Dis_isl_1.iloc[np.where(Dis_isl_1['Amtinloc.cur.']

In [6]:
#Dis_isl['Unnamed:1'].value_counts()

In [8]:
print(sum(merge.Disbursement_Drawdown_MYR))
print(sum(merge.Disbursement_Drawdown_Facility_Currency))
print(sum(merge.Cost_Payment_Principal_Repayment_MYR))
print(sum(merge.Cost_Payment_Principal_Repayment_Facility_Currency))

146014989.63
77249380.21000001
314584280.3699999
250909708.84000006


In [8]:
#merge.head(1)

In [9]:
#merge._merge.value_counts()

In [10]:
#merge.iloc[np.where(merge.Account==501015)]

In [11]:
#merge.head(1)

In [12]:
#appendfinal_ldb.tail(1)

In [9]:
merge['Account'] = merge['Account'].astype(str)

LDB_prev['Finance(SAP) Number'] = LDB_prev['Finance(SAP) Number'].astype(str)

LDB_prev.columns = LDB_prev.columns.str.replace("\n", "")
#LDB_prev.fillna(0, inplace=True)

LDB_prev['Cumulative Disbursement/Drawdown (Facility Currency)'].fillna(0,inplace=True)
LDB_prev['Cumulative Disbursement/Drawdown (MYR)'].fillna(0,inplace=True)
LDB_prev['Cumulative Cost Payment/Principal Repayment (Facility Currency)'].fillna(0,inplace=True) 
LDB_prev['Cumulative Cost Payment/Principal Repayment (MYR)'].fillna(0,inplace=True)

appendfinal_ldb = merge.merge(LDB_prev[['Finance(SAP) Number','EXIM Account No.','CIF Number',
                                              'Facility Currency',
                                              'Cumulative Disbursement/Drawdown (Facility Currency)',
                                              'Cumulative Disbursement/Drawdown (MYR)',
                                              'Cumulative Cost Payment/Principal Repayment (Facility Currency)',
                                              'Cumulative Cost Payment/Principal Repayment (MYR)']].drop_duplicates('Finance(SAP) Number',keep='first').rename(columns={'Finance(SAP) Number':'Account'}),on=['Account'],how='outer', suffixes=('_x', ''),indicator=True)

appendfinal_ldb['Disbursement_Drawdown_Facility_Currency'].fillna(0,inplace=True)
appendfinal_ldb['Disbursement_Drawdown_MYR'].fillna(0,inplace=True)
appendfinal_ldb['Cost_Payment_Principal_Repayment_Facility_Currency'].fillna(0,inplace=True) 
appendfinal_ldb['Cost_Payment_Principal_Repayment_MYR'].fillna(0,inplace=True)

appendfinal_ldb['Cumulative Disbursement/Drawdown (Facility Currency)'].fillna(0,inplace=True)
appendfinal_ldb['Cumulative Disbursement/Drawdown (MYR)'].fillna(0,inplace=True)
appendfinal_ldb['Cumulative Cost Payment/Principal Repayment (Facility Currency)'].fillna(0,inplace=True) 
appendfinal_ldb['Cumulative Cost Payment/Principal Repayment (MYR)'].fillna(0,inplace=True)

appendfinal2 = appendfinal_ldb#.fillna(0)

appendfinal2['Cumulative Disbursement/Drawdown (Facility Currency) New'] = appendfinal2['Disbursement_Drawdown_Facility_Currency'] +  appendfinal2['Cumulative Disbursement/Drawdown (Facility Currency)'] 
appendfinal2['Cumulative Disbursement/Drawdown (MYR) New'] = appendfinal2['Disbursement_Drawdown_MYR'] +  appendfinal2['Cumulative Disbursement/Drawdown (MYR)'] 

appendfinal2['Cumulative Cost Payment/Principal Repayment (Facility Currency) New'] = appendfinal2['Cost_Payment_Principal_Repayment_Facility_Currency'] +  appendfinal2['Cumulative Cost Payment/Principal Repayment (Facility Currency)'] 
appendfinal2['Cumulative Cost Payment/Principal Repayment (MYR) New'] = appendfinal2['Cost_Payment_Principal_Repayment_MYR'] +  appendfinal2['Cumulative Cost Payment/Principal Repayment (MYR)'] 

appendfinal2.sort_values('Disbursement_Drawdown_MYR', ascending=False, inplace=True)

C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_6924\3041795907.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  LDB_prev['Cumulative Disbursement/Drawdown (Facility Currency)'].fillna(0,inplace=True)
C:\Users\syahidhalid\AppData\Local\Temp\4\ipykernel_6924\3041795907.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we 

In [10]:
appendfinal_ldb._merge.value_counts()

_merge
right_only    251
both           76
left_only       0
Name: count, dtype: int64

In [11]:
print(sum(LDB_prev['Cumulative Disbursement/Drawdown (Facility Currency)']))
print(sum(LDB_prev['Cumulative Disbursement/Drawdown (MYR)']))
print(sum(LDB_prev['Cumulative Cost Payment/Principal Repayment (Facility Currency)']))
print(sum(LDB_prev['Cumulative Cost Payment/Principal Repayment (MYR)']))

2255096745.0199995
2831079012.4199996
1407605740.2385619
3128140862.560002


In [12]:
print(sum(appendfinal2['Cumulative Disbursement/Drawdown (Facility Currency)']))
print(sum(appendfinal2['Cumulative Disbursement/Drawdown (MYR)']))
print(sum(appendfinal2['Cumulative Cost Payment/Principal Repayment (Facility Currency)']))
print(sum(appendfinal2['Cumulative Cost Payment/Principal Repayment (MYR)']))

1127548372.5100002
1415539506.2099998
703802870.1192812
1564070431.2800007


In [17]:
#appendfinal2.head(1)

In [13]:
appendfinal3 = appendfinal2[['CIF Number','EXIM Account No.','Account',
'Curr.',
'Facility Currency',
'Type_of_Financing',
'Disbursement_Drawdown_Facility_Currency',
'Disbursement_Drawdown_MYR',
'Cumulative Disbursement/Drawdown (Facility Currency) New',
'Cumulative Disbursement/Drawdown (MYR) New',
'Cost_Payment_Principal_Repayment_Facility_Currency',
'Cost_Payment_Principal_Repayment_MYR',
'Cumulative Cost Payment/Principal Repayment (Facility Currency) New',
'Cumulative Cost Payment/Principal Repayment (MYR) New']]

In [14]:
#appendfinal3.to_excel(str(Location)+'\\02. Disbursement & Drawdown '+str(File[-8:])+' (python).xlsx', sheet_name='Export Worksheet', index = False)
appendfinal3.to_excel(r'C:\Users\syahidhalid\Syahid_PC\Analytics - FAD\00. Automation LDB\Data Source\202408\Working\02. Disbursement & Drawdown '+str(File[-8:])+' (python).xlsx', sheet_name='Export Worksheet', index = False)


In [ ]:
#appendfinal_ldb['Account'].value_counts()

In [ ]:
str(Location)+'\\02. Disbursement & Drawdown '+str(File[-8:])+' (python).xlsx'

In [ ]:
#Location[:-7]